# TF-IDF emoji

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv
from pathlib import Path
from collections import namedtuple, Counter

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import spacy
nlp = spacy.load('en_core_web_sm')

In [3]:
import random

In [99]:
meta = pd.read_csv("/content/drive/My Drive/NLP/Data/meta/metadata.txt",sep="\t")
meta
folds = meta.loc[0:20,]
folds 
MC_20 = ['❤','😂', '👍', '🙏', '🙌', '😘', '😍', '😊', '🔥', '👏', '👌', '💪', '👊', '😉', '🎉', '😎',"😁", "💯", "😜", "👀"]
folds = pd.DataFrame([meta.loc[i,] for i in range(len(meta[" emoji "])) if meta.loc[i,][" emoji "] in MC_20 ])
sum(folds[" len "])

308616

In [5]:
labs = {}
for i,fold in enumerate(folds[" emoji "]):
  labs[fold]=i
labs

{'❤': 4,
 '🎉': 11,
 '👀': 16,
 '👊': 1,
 '👌': 13,
 '👍': 15,
 '👏': 6,
 '💪': 0,
 '💯': 12,
 '🔥': 7,
 '😁': 9,
 '😂': 19,
 '😉': 10,
 '😊': 17,
 '😍': 18,
 '😎': 8,
 '😘': 14,
 '😜': 5,
 '🙌': 2,
 '🙏': 3}

##Twitter data

In [100]:


MC_20_train = namedtuple('MC_20_train',  ['text', 'lable'])
MC_20_test = namedtuple('MC_20_test',  ['text', 'lable'])
data_set_tr = []
data_set_te = []
for_embs_tr = []
for_embs_te = []

for i,fold in enumerate(folds["index "]):
    p = pd.read_csv("/content/drive/My Drive/NLP/Data" +"/"+ str(fold)+".csv",keep_default_na = False)
    for index,row in p.iterrows():
      rr = np.random.rand(1)[0]
      if rr < 0.66:
        if rr < 0.066: 
          mc_20_test = MC_20_test(
              row["Line"], row['Emoji']
          )
          for_embs_te.append(row["Line"])
          data_set_te.append(mc_20_test)
        else:
          mc_20_train = MC_20_train(
              row["Line"], row['Emoji']
          )
          for_embs_tr.append(row["Line"])
          data_set_tr.append(mc_20_train)

## MC20 data

In [74]:
import re
MC_20_train = namedtuple('MC_20_train',  ['text', 'lable'])
MC_20_test = namedtuple('MC_20_test',  ['text', 'lable'])
data_path = Path('/content/drive/MyDrive/NLP/NLP project/datasets/MC_20_dev.csv')

data_set_tr = []
data_set_te = []
for_embs_tr = []
for_embs_te = []

with data_path.open(encoding='utf-8', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      rr = np.random.rand(1)[0]
      if rr < 0.5:
        s = re.sub(r'http\S+', '', row['text'])
        s = re.sub(r'@\S+', '', s)
        s = re.sub(r'[^\w\s]','',s)
        s = re.sub(r'\w+_\S+',"", s)
        if rr < 0.05: 
          mc_20_test = MC_20_test(
              s, row['lable']
          )
          for_embs_te.append(s)
          data_set_te.append(mc_20_test)
        else:
          mc_20_train = MC_20_train(
              s, row['lable']
          )
          for_embs_tr.append(s)
          data_set_tr.append(mc_20_train)

In [101]:
len(for_embs_te)

20168

In [81]:
# lets learn the tf-idf 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
embeddings = vectorizer.fit_transform(for_embs_tr)

In [82]:
#lets find the test instance representation with the learned df and vocabulary
test_embs = vectorizer.transform(for_embs_te)

In [83]:
print(len(vectorizer.get_feature_names()))
print(embeddings.shape)

29227
(180039, 29227)


In [84]:
def cosine_similarity(query, pool, k=10):
    return np.argsort(-pool.dot(query.T).toarray().squeeze(-1))[0:k]

In [85]:
cosine_similarity(embeddings[0, :],embeddings)

array([     0,   8986,  99130,  36043, 153075,  63113,  27068, 126188,
       161935, 117098])

In [86]:
data_set_te[0].lable

'😜'

In [104]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def multi_label_accuracy(pred, gold):
    y = []
    for i in range(len(pred)):
        y.append(gold[i].lable)
    #print(y, pred)
    #precision, recall, f_score, n = precision_recall_fscore_support(y, pred, labels = MC_20, average=None)
    precision, recall, f_score, n = precision_recall_fscore_support(y, pred, average='macro')
    accuracy = accuracy_score(y, pred)
    return accuracy, precision, recall, f_score

In [102]:
def MRR(pred, i, labs):
  n = len(labs)
  preds = np.array([labs[data_set_tr[p].lable] for p in pred])
  x = np.tile(labs[data_set_te[i].lable], (n))
  rank = np.argmax(preds == x, axis=0)
  if rank == 0 and preds[0] != labs[data_set_te[i].lable]:
    return 0
  return (1/(rank+1))


In [103]:
def accuracy(pred, gold):
    assert len(pred) == len(gold)
    correct = 0
    total = 0
    for i in range(len(pred)):
        total += 1
        pp = pred[i]
        gg = data_set_te[i].lable
        if pp == gg:
            correct += 1
    return correct / total

In [ ]:
tfidf_pred = []
mrr = 0
for i, test_entry in enumerate(test_embs):
    sim = cosine_similarity(test_entry, embeddings, k=len(labs))
    mrr += MRR(sim, i, labs)
    most_similar = sim[0]
    tfidf_pred.append(data_set_tr[most_similar].lable)
    
print(accuracy(tfidf_pred, data_set_te))
print(multi_label_accuracy(tfidf_pred, data_set_te))


In [106]:
accuracy, precision, recall, f_score = multi_label_accuracy(tfidf_pred, data_set_te)
print(mrr/test_embs.shape[0])

0.07032892347966381


In [108]:
accuracy

0.04819636291364404

### Evaluation

In [ ]:
inv_voc = {}
for m in labs:
  inv_voc[labs[m]] = m
inv_voc

In [94]:
res = pd.DataFrame()
res["Emoji"] = MC_20
res["precision"] = np.round(precision,3)
res["recall"] = np.round(recall,3)
res["f_score"] = np.round(f_score,3)

In [96]:
res

,Emoji,precision,recall,f_score
0,❤,0.984,0.832,0.901
1,😂,0.967,0.799,0.875
2,👍,0.978,0.845,0.907
3,🙏,0.964,0.815,0.883
4,🙌,0.867,0.841,0.854
5,😘,0.977,0.765,0.858
6,😍,0.977,0.886,0.929
7,😊,0.949,0.860,0.902
8,🔥,0.996,0.914,0.953
9,👏,0.957,0.895,0.925


In [ ]:
test = ["I feel bliss"]
test_e = vectorizer.transform(test)
most_similar = cosine_similarity(test_e, embeddings, k=1)
data_set_tr[most_similar[0]].lable

'🙏'

In [60]:
import json
def make_uchr(code: str):
  return chr(int(code.lstrip("U+").zfill(8), 16))

lines_def = []
emojis = []
def_path = "/content/drive/My Drive/NLP/emojis.json"


with open(file = def_path) as f:
  def_data = pd.DataFrame.from_dict(json.load(f))
def_data = def_data.drop(["category","name", "senses"], axis=1)
def_data["shortcode"] = def_data["shortcode"].apply(str)
def_data["unicode"] = def_data["unicode"].apply(str)
def_data = def_data.loc[def_data["shortcode"] != "None"]

def_data["unicode"].index
for i,code in zip(def_data["unicode"].index,def_data["unicode"]):
  if len(code.split()) == 1:
    def_data["unicode"][i] = make_uchr(code)
#print(def_data["unicode"].values)
def_data

,keywords,definition,unicode,shortcode
5,[keycap],Keycap Digit Zero was approved as part of Unic...,U+0030 U+FE0F U+20E3,:zero:
140,"[m, circle]","The letter M in a circle, commonly used in to ...",Ⓜ,:m:
196,"[japanese, ?, “acceptable”, ideograph]",Japanese “acceptable” Button was approved as p...,🉑,:accept:
197,"[japanese, ?, “bargain”, ideograph]",Denotes a “good bargain” in Japanese. Japanese...,🉐,:ideograph_advantage:
201,"[japanese, ?, “open for business”, ideograph]",Means “work” in Japanese. This Emoji used to ...,🈺,:u55b6:
...,...,...,...,...
2383,"[sun, mountain, sunrise, morning]","The sun rising over a mountain range, early in...",🌄,:sunrise_over_mountains:
2384,"[night, star]","A night sky with stars, depicted on major plat...",🌃,:night_with_stars:
2385,"[umbrella, rain, clothing]","An umbrella that is closed, likely as a result...",🌂,:closed_umbrella:
2386,[fog],Foggy weather emoji. Shows fog covering a view...,🌁,:foggy:


In [61]:
def_data.reset_index()
MC_20 = ['❤','😂', '👍', '🙏', '🙌', '😘', '😍', '😊', '🔥', '👏', '👌', '💪', '👊', '😉', '🎉', '😎',"😁", "💯", "😜", "👀"]
eval = pd.DataFrame([def_data.iloc[i,] for i in range(len(def_data["unicode"])) if def_data.iloc[i,]["unicode"] in MC_20 ])

In [66]:
eval.columns = ["key", "Line", "Emoji", "sc"]
eval = eval.reset_index()

In [97]:
lines = []
preds = []
goldd = []
for i in range(eval.shape[0]):
  lines.append(eval["Line"][i].split(".")[0])
  test_e = vectorizer.transform([eval["Line"][i].split(".")[0]])
  goldd.append(eval["Emoji"][i])
  most_similar = cosine_similarity(test_e, embeddings, k=1)
  preds.append(data_set_tr[most_similar[0]].lable)

dff = pd.DataFrame()
dff["Gold"] = goldd
dff["Tf-idf"] = preds
dff["Line"] = lines
dff.to_markdown()

'|    | Gold   | Tf-idf   | Line                                                                                        |\n|---:|:-------|:---------|:--------------------------------------------------------------------------------------------|\n|  0 | 🙏     | 👀       | Two hands placed firmly together, meaning please\xa0or\xa0thank you\xa0in Japanese culture           |\n|  1 | 🙌     | 👍       | Two hands raised in the air, celebrating success or another joyous event                    |\n|  2 | 😜     | 🎉       | A face showing a stuck-out tongue, winking at the same time                                 |\n|  3 | 😘     | 😘       | An emoji face blowing a\xa0kiss;\xa0but officially called “Face Throwing A Kiss”                  |\n|  4 | 😎     | 👀       | A face smiling and wearing dark sunglasses that is\xa0used to denote a sense of cool           |\n|  5 | 😍     | 👀       | A face with hearts instead of eyes, or Heart Eyes Emoji as it is generally known            |\n|  6 | 😊     | 😁 

In [98]:
dff

,Gold,Tf-idf,Line
0,🙏,👀,"Two hands placed firmly together, meaning plea..."
1,🙌,👍,"Two hands raised in the air, celebrating succe..."
2,😜,🎉,"A face showing a stuck-out tongue, winking at ..."
3,😘,😘,An emoji face blowing a kiss; but officially c...
4,😎,👀,A face smiling and wearing dark sunglasses tha...
5,😍,👀,"A face with hearts instead of eyes, or Heart E..."
6,😊,😁,"A smiling face, with smiling eyes and rosy cheeks"
7,😉,👌,A classic winky emoji; winking and smiling
8,😂,💯,A laughing emoji which at small sizes is often...
9,😁,😁,A version of the grinning face showing smiling...
